In [ ]:
import pandas as pd
import numpy as np
import mlflow

from mlflow.models.signature import infer_signature#
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    fbeta_score,
    log_loss,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
    confusion_matrix,
    auc,
    brier_score_loss,
)


from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
experiment_name = f"/cancer_classification_training"
run_name = "cancer_classification_training" + str(pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"))
full_model_name = "cancer_classification_model"

LABEL = "Target"

In [ ]:
# defining these up front to make mlflow experiments optional
mlflow_metrics = {} # a place to pop new metrics into
mlflow_figures = {} # a place to pop new matplotlib/sns figures into
mlflow_tables = {} # a place to pop new dataframes into (keep it small!)
mlflow_artifact_paths = []

In [ ]:
def push_metric(metric_name: str, metric_value: any) -> None:
    print(f"Adding metric {metric_name}: {metric_value}")
    mlflow_metrics[metric_name] = metric_value

def push_figure(figure_name: str, figure_value: any):
    print(f"Adding figure {figure_name}: {figure_value}")
    mlflow_figures[figure_name] = figure_value

def push_table(table_name: str, table_value: any):
    print(f"Adding table {table_name}: {table_value}")
    mlflow_tables[table_name] = table_value

def push_artifact(file_path: str):
    print(f"Adding artifact {file_path}")
    mlflow_artifact_paths.append(file_path)

In [ ]:
#Get clean data

def get_clean_data(filename: str) -> pd.DataFrame:
    # Load the dataset
    data = pd.read_csv(filename)
    return data


def get_train_data(data: pd.DataFrame):
    """Get training data from the cleaned dataset."""
    # Split the data into features and Target
    X = data
    y = data['Target']

    # Split the data into training and testing sets
    # Stratify to ensure both classes are represented in train/test splits
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    train_ds, val_ds = train_test_split(
        X_train, test_size=0.1, random_state=20000, shuffle=True
    )
    X_train = X_train.drop(columns=['Target'])
    X_val = val_ds.drop(columns=['Target'])
    X_test = X_test.drop(columns=['Target'])
    y_val = val_ds['Target']
     


    return X_train, X_test, y_train, y_test, X_val, y_val

In [ ]:
# Define the model metrics

def calculate_metrics(y_true, y_pred, y_proba):
    """Calculate model metrics."""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    logloss = log_loss(y_true, y_proba)

    return accuracy, precision, recall, f1, logloss

In [ ]:
def get_model_metrics(
    model: lgb.Booster, X_test: pd.DataFrame, y_test: pd.Series
) -> dict:
    y_pred_probablity = model.predict(X_test)
    pred_threshold = np.percentile(y_pred_probablity, 60)
    y_pred_label = np.where(y_pred_probablity >= pred_threshold, 1, 0)

    metrics = {
        "logloss": log_loss(y_test, y_pred_probablity),
        "pred_threshold": pred_threshold,
        "accuracy": accuracy_score(y_test, y_pred_label),        
        "f1_score": f1_score(y_test, y_pred_label),
        "f05_score": fbeta_score(y_test, y_pred_label, beta=0.5),
        "precision_score": precision_score(y_test, y_pred_label),
        "recall_score": recall_score(y_test, y_pred_label),
        "roc_auc_score": roc_auc_score(y_test, y_pred_probablity),
        "y_real_mean": y_test.mean(),
        "y_pred_mean": y_pred_probablity.mean(),
        "iterations": model.current_iteration(),
    }
    return metrics

In [ ]:
mlflow.set_registry_uri("databricks-uc")
mlflow.set_experiment(experiment_name)
run = mlflow.start_run(run_name=run_name, description=f"Building a model for {experiment_name}")
print(f"creating run called {run.info.run_name}")

In [ ]:
clean_df = get_clean_data('../data/clean.csv')
push_metric("count_initial", clean_df.shape[0])

In [ ]:
X_train, X_test, Y_train, Y_test, X_val, Y_val = get_train_data(clean_df)


In [ ]:
unsupported_cols = X_train.select_dtypes(exclude=['int', 'float', 'bool']).columns
print("Columns with unsupported data types:", unsupported_cols)

In [ ]:
evals_result = {}

In [ ]:
def eval_logloss(y_hat, data):
    y_true = data.get_label()
    return "logloss", log_loss(y_true, y_hat), False

In [ ]:

# Define constants
PRIMARY_METRIC = "logloss"  # or "f1"
ESTIMATORS = 50 
EARLY_STOP = 5
LEARNING_RATE = 0.045
CATEGORICAL_FEATURES = []
# Prepare datasets
# Assuming X_train, y_train, X_valid, y_valid are predefined
lgb_train_final = lgb.Dataset(
    X_train,
    label=Y_train,
    feature_name=list(X_train.columns),
    categorical_feature=CATEGORICAL_FEATURES
)

lgb_valid = lgb.Dataset(X_val, label=Y_val, reference=lgb_train_final)

# lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train_final)

# Define parameters
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": PRIMARY_METRIC,
    "num_threads": 16,
    "learning_rate": LEARNING_RATE,
}

# Define callbacks
callbacks = [
    lgb.early_stopping(stopping_rounds=EARLY_STOP),
    lgb.log_evaluation(period=10),  # Adjust the period as needed
]

final_model = lgb.train(
    params=params,
    train_set=lgb_train_final,
    valid_sets=lgb_valid, 
    num_boost_round=ESTIMATORS,
    feval=eval_logloss,
    callbacks=callbacks
)

print("Final model built")

In [ ]:
metrics = get_model_metrics(
    final_model, X_test, Y_test)


In [ ]:
for (metric_name, metric_value) in metrics.items():
    push_metric(metric_name, metric_value)

Plot: Receiver Operating Characteristic (ROC) Curve

In [ ]:

y_pred_proba = final_model.predict(X_test)
fpr, tpr, thresholds = roc_curve(Y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')

auc_legend = (
    "AUC Score Interpretation:\n"
    "1.0       : Perfect model\n"
    "0.9–0.99  : Excellent\n"
    "0.8–0.9   : Good\n"
    "0.7–0.8   : Fair\n"
    "0.5       : No better than random\n"
    "< 0.5     : Worse than random"
)
plt.gca().text(
    1.05, 0.5, auc_legend,
    transform=plt.gca().transAxes,
    fontsize=10,
    verticalalignment='center',
    bbox=dict(boxstyle="round,pad=0.5", edgecolor="gray", facecolor="#f9f9f9")
)
plt.legend(loc="lower right")
plt.tight_layout()
plt.show()

push_figure("receiver_operating_characteristic.png", plt.gcf())

### Plot: Kernel Density Estimate (KDE)

In [ ]:
plt.figure(figsize=(8, 5))
for label in Y_test.unique():
    subset = final_model.predict(X_test[Y_test == label])
    sns.kdeplot(subset, label=f"Target={label}", fill=True)
plt.title("Kernel Density Estimate (KDE) of Predicted Probabilities by True Class")
plt.xlabel("Predicted Probability")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

push_figure("kde_plot.png", plt.gcf())

Plot: Confusion Matrix

In [ ]:

conf_matrix = confusion_matrix(Y_test, np.where(y_pred_proba >= metrics["pred_threshold"], 1, 0))
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])

plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()

plt.show()

push_figure("confusion_matrix.png", plt.gcf())


In [ ]:
ax_importance_gain = lgb.plot_importance(final_model, max_num_features=50, importance_type="gain")
ax_importance_gain.figure.tight_layout()
plt.show()

push_figure("feature_importance_gain.png", ax_importance_gain.figure)

In [ ]:
print(f"registering model as: {full_model_name}")
autolog_run = mlflow.last_active_run()
model_uri = "runs:/{}/models/final_model".format(autolog_run.info.run_id)

# create a signature for the model so that mlflow knows the inputs and outputs (we need this because we're not using mlflow.autolog)
X_train_sample = pd.DataFrame(X_train[:10], columns=X_train.columns)
signature = infer_signature(X_train_sample, final_model.predict(X_train_sample))

mlflow.lightgbm.log_model(
    final_model,
    registered_model_name=full_model_name,
    signature=signature
)
# this simply logs metadata about the dataset (not the data itself)
mlflow.log_input(mlflow.data.from_pandas(X_test), context="test")
mlflow.log_input(mlflow.data.from_pandas(X_train), context="train")

In [0]:
if mlflow.active_run() == None:
    raise Exception("A run hasn't been started")


# ADD other metrics
for (metric_name, metric_value) in mlflow_metrics.items():
    print(f"logging other metric: {metric_name}=metric_value")
    mlflow.log_metric(metric_name, metric_value)

for (figure_name, figure) in mlflow_figures.items():
    print(f"logging figure: {figure_name}")
    mlflow.log_figure(figure, figure_name)

for (table_name, table_df) in mlflow_tables.items():
    print(f"logging table: {table_name}")
    mlflow.log_table(table_df, table_name)

In [ ]:
mlflow.end_run()